# Hartree-Fock - Kinetic Energy

The Fock operator is 

$$\hat{F}(\vec{r}) = \hat{h}(\vec{r}) + \sum_{j=1}^{N} \left( \hat{J}_j(\vec{r}) - \hat{K}_j(\vec{r}) \right)$$ 

where 

$$\hat{h} = \hat{T} + \hat{V}_{eN}$$

In this notebook we will be working with the kinetic energy matrix, 

$$\boxed{T_{ij} = \int d^3r \varphi_i^*(\vec{r}) \hat{T} \varphi_j(\vec{r})}$$



In [1]:
import numpy as np 
from Utils import * # Import functions from the previous notebook: Primitive Class and Overlap

The matrix elements of kinetic energy are found analogously to the overlap matrix

$$T_{ij} = -\frac12 \int d^3r \varphi_i^*(\vec{r}) \nabla^2 \varphi_j(\vec{r})$$

Notice that 

$$\nabla^2_i \phi_\nu(\vec{r}) = 2 \alpha_\nu \left[ -3 + 2 \alpha_\nu (\vec{r}-\vec{r}_\nu)^2\right] \phi_\nu(\vec{r})$$

So, 

\begin{align*}
T_{ij} &= -\frac12 \int d^3r \sum_{\mu=1}^K c_{i\mu} \phi_{\mu}(\vec{r}) \nabla^2 \sum_{\nu=1}^K c_{\nu j} \phi_{\nu}(\vec{r}) \\
 &= -\frac12 \int d^3r \sum_{\mu,\nu=1}^K 2 \alpha_\nu c_{i\mu} c_{\nu j} \phi_{\mu}(\vec{r}) \left[ -3 + 2\alpha_\nu (\vec{r}-\vec{r}_\nu)^2 \right] \phi_{\nu}(\vec{r}) \\
 &= - \sum_{\mu,\nu=1}^K \alpha_\nu c_{i\mu} c_{\nu j} \left[ -3 \int d^3r  \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r}) + 2\alpha_\nu\int d^3r (\vec{r}-\vec{r}_\nu)^2 \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r}) \right] \\
 &= 3 \sum_{\mu,\nu=1}^K \alpha_\nu c_{i\mu} c_{\nu j} s_{\mu\nu} - 2  \sum_{\mu,\nu=1}^K \alpha_\nu^2 c_{i\mu} c_{\nu j} s_{\mu\nu}' 
\end{align*}

where 

$$s_{\mu\nu} = \left( \frac{\pi}{\alpha_\mu + \alpha_\nu}\right)^{3/2}  \left( \frac{4 \alpha_\mu \alpha_\nu}{\pi^2}\right)^{3/4} \exp\left\{ - \frac{\alpha_\mu \alpha_\nu}{\alpha_\mu + \alpha_\nu} (\vec{r}_\mu - \vec{r}_\nu)^2\right\}$$

is the overlap matrix, and

$$s_{\mu\nu}' = \int d^3r (\vec{r}-\vec{r}_\nu)^2 \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})$$

is the additional term, which can be shown to be equal to 

$$s_{\mu\nu}' = \left[ \frac32 \frac{1}{\alpha_\mu+\alpha_\nu} + \frac{\alpha_\nu^2}{(\alpha_\mu + \alpha_\nu)^2} (\vec{r}_\mu-\vec{r}_\nu)^2 \right] s_{\mu\nu}$$

Therefore, the kinetic energy term is 

$$T_{ij} = 3 \sum_{\mu,\nu=1}^K \alpha_\nu c_{i\mu} c_{\nu j} s_{\mu\nu} - 2  \sum_{\mu,\nu=1}^K \alpha_\nu^2 c_{i\mu} c_{\nu j} \left[ \frac32 \frac{1}{\alpha_\mu+\alpha_\nu} + \frac{\alpha_\nu^2}{(\alpha_\mu + \alpha_\nu)^2} (\vec{r}_\mu-\vec{r}_\nu)^2 \right] s_{\mu\nu}$$

$$\boxed{ T_{ij} = 3 \sum_{\mu,\nu=1}^K \alpha_\nu c_{i\mu} c_{\nu j} s_{\mu\nu} - 2  \sum_{\mu,\nu=1}^K \alpha_\nu^2 c_{i\mu} c_{\nu j} \left[ \frac32 \frac{1}{\alpha_\mu+\alpha_\nu} +  \left(\frac{\alpha_\nu x_\nu + \alpha_\mu x_\mu}{\alpha_\nu + \alpha_\mu} - x_\nu \right)^2 \right] s_{\mu\nu} }$$

In [2]:
def kinetic(molecule):
            
    n_basis = len(molecule) # Number of atoms 
    T = np.zeros((n_basis,n_basis)) # Overlap between the atomic orbitals
            
    # The first loop runs over all orbitals
    for i in range(n_basis):
        for j in range(n_basis):
            
            n_primitives_i = len(molecule[i]) # Number of primitives in which orbital i is decomposed
            n_primitives_j = len(molecule[j]) # Number of primitives in which orbital j is decomposed
            
            # The second loop runs over the primitives of the orbitals to be overlap
            for k in range(n_primitives_i): 
                for l in range(n_primitives_j):
                    
                    N = molecule[i][k].A * molecule[j][l].A # Product of primitive normalization constants
                    p = molecule[i][k].α + molecule[j][l].α # α_μ + α_ν
                    q = molecule[i][k].α * molecule[j][l].α / p # α_μ * α_ν / α_μ + α_ν
                    Q = molecule[i][k].coords - molecule[j][l].coords # r_μ - r_ν 
                    Q2 = np.dot(Q,Q) # (r_μ - r_ν)^2
                    
                    a = (molecule[i][k].α/p)**2 # (α_μ / α_μ + α_ν)^2
                    
                        
                    
                    S = N * molecule[i][k].coeff * molecule[j][l].coeff * np.exp(-q*Q2) * (np.pi/p)**(3/2) # Overlap matrix element
                    
                    T[i,j] += 3*molecule[j][l].α*S - 2*(molecule[j][l].α)**2 * (1.5/p + a*Q2) * S # Kinetic energy matrix element
                    
    return T 

In [3]:
# STO-3G BASIS FOR 1S ORBITALS
# Primitive gaussians of the first hydrogen
H1_pgaussian1a = primitive_gaussian(0.3425250914E+01,0.1543289673E+00,[0,0,0])
H1_pgaussian1b = primitive_gaussian(0.6239137298E+00,0.5353281423E+00,[0,0,0])
H1_pgaussian1c = primitive_gaussian(0.1688554040E+00,0.4446345422E+00,[0,0,0])

# Primitive gaussians of the second hydrogen
H2_pgaussian1a = primitive_gaussian(0.3425250914E+01,0.1543289673E+00,[1.4,0,0])
H2_pgaussian1b = primitive_gaussian(0.6239137298E+00,0.5353281423E+00,[1.4,0,0])
H2_pgaussian1c = primitive_gaussian(0.1688554040E+00,0.4446345422E+00,[1.4,0,0])

# Atomic orbitals
H1_1s = [H1_pgaussian1a, H1_pgaussian1b, H1_pgaussian1c]
H2_1s = [H2_pgaussian1a, H2_pgaussian1b, H2_pgaussian1c]

# Molecule
molecule = [H1_1s, H2_1s] 

# Overlap matrix
print("Overlap: \n",overlap(molecule))

# Kinetic energy matrix
print("Kinetic Energy: \n",kinetic(molecule))

Overlap: 
 [[1.         0.65931821]
 [0.65931821 1.        ]]
Kinetic Energy: 
 [[0.76003188 0.23645466]
 [0.23645466 0.76003188]]
